# LangGraph Tutorial: Graph Construction

**Objective:** Build the agentic workflow that connects LLM and tools.

**What We'll Build:**
```
START → Agent (LLM) → Router → [Tools or END]
           ↑                        ↓
           └────────────────────────┘
```

## Setup Mermaid

Define render_mermaid function.

In [5]:
# Mermaid helper for visualization
def render_mermaid(diagram_code, width=400):
    '''Helper function to render Mermaid diagrams using mermaid.ink'''
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

## Recreate Tools

Copy the tools from Notebook 2.

In [6]:
from langchain_core.tools import tool

In [7]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the conversion result including the exchange rate
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )
    
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5 for 8.5%)
        tenure_months: Loan tenure in months
        currency: Currency code for display
    
    Returns:
        A string with EMI calculation details
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")
print(f"   • {currency_converter.name}")
print(f"   • {emi_calculator.name}")

✅ Tools defined
   • currency_converter
   • emi_calculator


## Initialize LLM with Tools

Bind tools to the LLM so it knows they're available.

In [16]:
import os
from dotenv import load_dotenv

# Load environment
load_dotenv("../../.env")

True

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [18]:
# Create base LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    max_tokens=1024,
    project=os.getenv("GOOGLE_PROJECT_ID"),
    location=os.getenv("GOOGLE_REGION")
)

# Define tools list
tools = [currency_converter, emi_calculator]

# Bind tools to LLM
# This tells the LLM: "You can call these tools when needed"
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")
print(f"   Model: gemini-2.5-pro")
print(f"   Tools bound: {len(tools)}")

✅ LLM initialized with tools
   Model: gemini-2.5-pro
   Tools bound: 2


## Define Graph Components

### Component 1: Agent Node

The agent calls the LLM and returns its decision.

In [19]:
# Core imports

from langgraph.graph import MessagesState


In [20]:
def call_llm(state: MessagesState):
    """
    Agent node that invokes the LLM.
    
    The LLM analyzes the conversation and decides to either:
    1. Call tools (returns AIMessage with tool_calls)
    2. Provide final response (returns AIMessage with content)
    
    Args:
        state: Current graph state with message history
        
    Returns:
        Dictionary with new messages to append
    """
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

print("✅ Agent node (call_llm) defined")

✅ Agent node (call_llm) defined


### Component 2: Router Function

Decides the next step based on LLM's response.

In [21]:
from typing import Literal

In [22]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    """
    Router that determines next node.
    
    Checks the last message:
    - If it has tool_calls → route to "tools" node
    - Otherwise → route to END (finish)
    
    Args:
        state: Current graph state
        
    Returns:
        Either "tools" string or END constant
    """
    last_message = state["messages"][-1]
    
    # If LLM made tool calls, execute them
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return END

print("✅ Router (should_continue) defined")

✅ Router (should_continue) defined


## Build the Graph

Connect all components into an executable workflow.

In [23]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

In [24]:
print("Building StateGraph...\n")

# Initialize graph with MessagesState
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("agent", call_llm)           # LLM decision-maker
workflow.add_node("tools", ToolNode(tools))    # Tool executor

# Add edges
workflow.add_edge(START, "agent")  # Always start with agent

# Add conditional edge from agent
workflow.add_conditional_edges(
    "agent",           # From this node
    should_continue,   # Use this function to decide
    {
        "tools": "tools",  # If returns "tools", go to tools node
        END: END            # If returns END, finish
    }
)

# After tools execute, loop back to agent
workflow.add_edge("tools", "agent")

# Compile into executable app
app = workflow.compile()

print("✅ Graph compiled successfully!")
print("\nGraph Structure:")
print("  START → agent → [router decision]")
print("            ↓           ↓")
print("          END  ←──  tools")
print("                      ↓")
print("                    agent (loop)")

Building StateGraph...

✅ Graph compiled successfully!

Graph Structure:
  START → agent → [router decision]
            ↓           ↓
          END  ←──  tools
                      ↓
                    agent (loop)


## Visualize the Graph

Generate and render the Mermaid diagram.

In [25]:
# Get Mermaid diagram from graph
mermaid_diagram = app.get_graph().draw_mermaid()

# Render it
render_mermaid(mermaid_diagram)

## Understanding the Flow

**Key Points:**

1. **State = Messages List**
   - Every node reads `state["messages"]`
   - Every node returns `{"messages": [new_messages]}`
   - LangGraph auto-appends new messages to state

2. **The Cycle Enables Reasoning**
   ```
   Agent → Tools → Agent → Tools → Agent → END
   ```
   - Agent can call tools multiple times
   - Each loop refines understanding
   - Terminates when agent has enough info

3. **Router Controls Flow**
   - Checks if LLM wants to use tools
   - Routes to `tools` or `END` accordingly
   - Purely conditional - no hardcoded logic

## Test Basic Execution

Quick sanity check before detailed testing in next notebook.

In [26]:
from langchain_core.messages import HumanMessage

In [27]:
# Create test state
test_state = {"messages": [HumanMessage(content="Convert 100 USD to EUR")]}

# Execute
result = app.invoke(test_state)

# Display result
print("Test Query: Convert 100 USD to EUR\n")
print("Agent Response:")
print("=" * 70)
final_message = result["messages"][-1]
print(final_message.content)
print("\n✅ Graph execution successful!")

Test Query: Convert 100 USD to EUR

Agent Response:
100 USD is equal to 92 EUR.

✅ Graph execution successful!


---

## ✅ Graph Constructed!

**Built:**
- ✅ Agent node (LLM decision-maker)
- ✅ Router function (conditional logic)
- ✅ Tools node (executes tools in parallel)
- ✅ Graph with cycle (agent ↔ tools loop)

**Verified:**
- ✅ Graph compiles without errors
- ✅ Mermaid visualization renders
- ✅ Basic execution works

**Next:** Notebook 4 - Single Tool Execution (detailed execution analysis)